<a href="https://www.kaggle.com/code/jorgeluiscruzquispe/5-vectorsandtables?scriptVersionId=161587855" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <center>F5 - Earth Engine Fundamentals and Applications - EEFA</center>

---

In [1]:
!pip install geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 68.0 MB/s eta 0:00:00:00:01
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11122 sha256=accd013ab8b3cdc1ecdbbbb51715b91549faaa41ef8795e6efe13f705a7ff5c7
  Stored in directory: /root/.cache/pip/wheels/04/24/fe/140a94a7f1036003ede94579e6b4227fe96c840c6f4dcbe307
Successfully built pyperclip


In [2]:
import geemap.foliumap as geemap
import ee

In [3]:
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AfJohXk4LwzY6CVhoijUy5b1EBLo9kpwkXpQ5JuOQV3YJQURvjKSrsdQzNw



Successfully saved authorization token.


In [8]:
Map = geemap.Map()

# Import the Census Tiger Boundaries from GEE.
tiger = ee.FeatureCollection('TIGER/2010/Blocks')

# Add the new feature collection to the map, but do not display.
Map.addLayer(tiger, {'color': 'black'}, 'Tiger', False)

# Map.centerObject(tiger, 14)

Map

In [9]:
Map = geemap.Map()

# Assign the feature collection to the variable sfNeighborhoods.
sfNeighborhoods = ee.FeatureCollection('projects/gee-book/assets/F5-0/SFneighborhoods')

# Print the size of the feature collection.
# (Answers the question how many features?)
# print(sfNeighborhoods.size())

point = ee.Geometry.Point([-122.468719,37.755787])

usfNeighborhood = sfNeighborhoods.filterBounds(point)

Map.addLayer(sfNeighborhoods, {'color': 'blue'}, 'sfNeighborhoods', opacity = 0.75)

Map.addLayer(usfNeighborhood, {'color': 'black'}, 'neightbor', opacity = 0.75)

Map.addLayer(point, {'color': 'red'}, 'punto')

Map.centerObject(point, 14)

Map

In [10]:
Map = geemap.Map()

sfNeighborhoods = ee.FeatureCollection('projects/gee-book/assets/F5-0/SFneighborhoods')

tiger = ee.FeatureCollection('TIGER/2010/Blocks')

point = ee.Geometry.Point([-122.468719,37.755787])

usfNeighborhood = sfNeighborhoods.filterBounds(point)

usfTiger = tiger.filterBounds(usfNeighborhood)

Map.addLayer(tiger, {'color': 'blue'}, 'Tiger', opacity = 0.5)

Map.addLayer(usfTiger, {"color": "black"}, 'usf_Tiger')

Map.centerObject(point, 14)

Map.addLayer(point, {'color': 'red'}, 'punto')

Map

### Filter by Feature (Attribute) Properties


In [11]:
print(usfTiger)

ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.loadTable",
          "arguments": {
            "tableId": {
              "constantValue": "TIGER/2010/Blocks"
            }
          }
        }
      },
      "filter": {
        "functionInvocationValue": {
          "functionName": "Filter.intersects",
          "arguments": {
            "leftField": {
              "constantValue": ".all"
            },
            "rightValue": {
              "functionInvocationValue": {
                "functionName": "Feature",
                "arguments": {
                  "geometry": {
                    "functionInvocationValue": {
                      "functionName": "Collection.geometry",
                      "arguments": {
                        "collection": {
                          "functionInvocationValue": {
     

In [12]:
Map = geemap.Map()

point = ee.Geometry.Point([-122.468719,37.755787])

# Filter for census blocks by housing units.
housing10_l250 = usfTiger.filter(ee.Filter.lt('housing10', 250))

housing10_g50_l250 = housing10_l250.filter(ee.Filter.gt('housing10', 50))

Map.addLayer(housing10_g50_l250, {'color': 'Magenta'}, 'housing')

Map.centerObject(point, 14)

Map.addLayer(point, {'color': 'red'}, 'punto')

Map

In [13]:
housing_area = housing10_g50_l250.geometry().area()
print(f'housing_area: {(housing_area.getInfo()):.2f} m2');

housing_area: 2372275.01 m2


In [14]:
housing10_mean = usfTiger.reduceColumns(reducer = ee.Reducer.mean(),selectors = ['housing10'])

mean = housing10_mean.getInfo()['mean']

print(f'housing10_mean {mean:.2f}')

housing10_mean 54.30


In [15]:
Map = geemap.Map()

point = ee.Geometry.Point([-122.468719,37.755787])


# Import the Landsat 8 TOA image collection.
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')

# Get the least cloudy image in 2015.
image = ee.Image(landsat8.filterBounds(point).filterDate('2015-01-01', '2015-12-31').sort('CLOUD_COVER').first())

nir = image.select('B5')

red = image.select('B4')

ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

ndviParams = {"min": -1,"max": 1,"palette": ['blue', 'white', 'green']}

ndviUSFblocks = ndvi.clip(housing10_g50_l250)

Map.addLayer(ndviUSFblocks, ndviParams, 'NDVI image')

Map.addLayer(ee.Image().paint(housing10_g50_l250, 1, 1.5), {"palette": "red"}, "housing10_g50_l250")

Map.centerObject(point, 14)

Map

In [16]:
Map = geemap.Map()

# Reduce image by feature to compute a statistic e.g. mean, max, min etc.
ndviPerBlock = ndviUSFblocks.reduceRegions(collection = housing10_g50_l250,reducer = ee.Reducer.mean(), scale = 30)

Map.addLayer(ndviPerBlock, {"color": "red"}, "ndviPerBlock")

Map.centerObject(point, 14)

Map

In [17]:
display(housing10_g50_l250)
ndviPerBlock

Name,Type,Description
blockce,STRING,2010 Census tabulation block number
blockid10,STRING,"Block identifier: a concatenation of 2010 Census state Federal Information Processing Standards (FIPS) code, county FIPS code, census tract code, and tabulation block number"
countyfp10,STRING,County FIPS code
housing10,DOUBLE,2010 Census number of housing units
partflg,STRING,Partial block flag
pop10,DOUBLE,Population total as of 2010 census
statefp10,STRING,2010 Census state FIPS code
tractce10,STRING,2010 Census tract code


In [18]:
Map = geemap.Map()

sfNeighborhoods = ee.FeatureCollection('projects/gee-book/assets/F5-0/SFneighborhoods')

tiger = ee.FeatureCollection('TIGER/2010/Blocks')

point = ee.Geometry.Point([-122.468719,37.755787])

usfNeighborhood = sfNeighborhoods.filterBounds(point)

usfTiger = tiger.filterBounds(usfNeighborhood)

ndviPerBlock = ndviPerBlock.select(['blockid10', 'mean'])
# print('ndviPerBlock', ndviPerBlock)
ndviPerBlockSorted = ndviPerBlock.sort('mean', False)
ndviPerBlockSortedFirst = ee.Feature(ndviPerBlock.sort('mean', False).first()) 
# Sort by NDVI mean in descending order.
# Select the block with the highest NDVI.
# print('ndviPerBlockSortedFirst', ndviPerBlockSortedFirst)

# Now filter by block and show on map! 
GreenHousing = usfTiger.filter(ee.Filter.eq('blockid10', '#####')) # < Put your id here prepend a 0!
Map.addLayer(GreenHousing, {'color': 'red'}, 'Green Housing!')

Map.centerObject(point, 13)

Map

## Chapter F5.1: Raster/Vector Conversions


In [19]:
Map = geemap.Map()

# Load raster (elevation) and vector (colombia) datasets.
elevation = ee.Image('USGS/GMTED2010').rename('elevation')
colombia = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level0').filter(ee.Filter.equals('ADM0_NAME', 'Colombia'))

# Display elevation image.
Map.centerObject(colombia, 6)
Map.addLayer(elevation, {"min": 0,"max": 4000}, 'Elevation')

Map

In [20]:
Map = geemap.Map()

# Initialize image with zeros and define elevation zones.
zones = ee.Image(0)\
            .where(elevation.gt(100), 1)\
            .where(elevation.gt(200), 2)\
            .where(elevation.gt(500), 3)

# Mask pixels below sea level (<= 0 m) to retain only land areas.
# Name the band with values 0-3 as 'zone'.
zones = zones.updateMask(elevation.gt(0)).rename('zone')

Map.addLayer(zones, {
    "min": 0,
    "max": 3,
    "palette": ['white', 'yellow', 'lime', 'green'],
    "opacity": 0.7
}, 'Elevation zones');

Map

In [21]:
Map = geemap.Map()

projection = elevation.projection()
scale = elevation.projection().nominalScale()

elevationVector = zones.reduceToVectors(
    geometry = colombia.geometry(),
    crs = projection,
    scale = 1000, # scale
    geometryType = 'polygon',
    eightConnected = False,
    labelProperty = 'zone',
    bestEffort = False,
    maxPixels = 1e13,
    tileScale = 3 # In case of error.
)

# print(elevationVector.limit(10))

elevationDrawn = elevationVector.draw(
    color = 'blue',
    strokeWidth = 1
)

Map.addLayer(elevationDrawn, {}, 'Elevation zone polygon')

Map.centerObject(colombia, 6)

Map

In [22]:
Map = geemap.Map()

zonesSmooth = zones.focalMode(4, 'square')

zonesSmooth = zonesSmooth.reproject(projection.atScale(scale))

Map.addLayer(zonesSmooth, {
    "min": 1,
    "max": 3,
    "palette": ['yellow', 'lime', 'green'],
    "opacity": 0.7
}, 'Elevation zones (smooth)')

elevationVectorSmooth = zonesSmooth.reduceToVectors(
    geometry = colombia.geometry(),
    crs = projection,
    scale = scale,
    geometryType = 'polygon',
    eightConnected = False,
    labelProperty = 'zone',
    bestEffort = False,
    maxPixels = 1e13,
    tileScale = 3
)

smoothDrawn = elevationVectorSmooth.draw(
    color = 'black',
    strokeWidth = 1
)

Map.addLayer(smoothDrawn, {}, 'Elevation zone polygon (smooth)')

Map.centerObject(colombia, 6)

Map

In [23]:
Map = geemap.Map()

geometry = ee.Geometry.Polygon([
    [-89.553, -0.929],
    [-89.436, -0.929],
    [-89.436, -0.866],
    [-89.553, -0.866],
    [-89.553, -0.929]])

# To zoom into the area, un-comment and run below
# Map.centerObject(geometry,12);
Map.addLayer(geometry, {"color": "green"}, 'Areas to extract points')

elevationSamples = elevation.sample(
    region = geometry,
    projection = projection,
    scale = scale,
    geometries = True,
)

Map.addLayer(elevationSamples, {"color": "orange"}, 'Points extracted')

# Add three properties to the output table: 
# 'Elevation', 'Longitude', and 'Latitude'.

# def coordinates_extract(feature):
#     geom = feature.geometry().coordinates()
#     return ee.Feature(None, 
#         Elevation = ee.Number(feature.get('elevation')),
#         Long = ee.Number(geom.get(0)),
#         Lat = ee.Number(geom.get(1)))



# elevationSamples = elevationSamples.map(coordinates_extract)
 

# Export as CSV.
# Export.table.toDrive(
#     collection = elevationSamples,
#     description = 'extracted_points',
#     fileFormat = 'CSV'
# )

Map.centerObject(elevationSamples, 13)

Map

In [24]:
Map = geemap.Map()

elevationSamplesStratified = zones.stratifiedSample(
    numPoints = 10,
    classBand = 'zone',
    region = geometry,
    scale = scale,
    projection = projection,
    geometries = True
)

Map.addLayer(elevationSamplesStratified, {}, 'Stratified samples')

Map.centerObject(elevationSamplesStratified, 13)

Map

In [25]:
# Read input data.
# Note: these datasets are periodically updated.
# Consider searching the Data Catalog for newer versions.
gfc = ee.Image('UMD/hansen/global_forest_change_2020_v1_8')
wdpa = ee.FeatureCollection('WCMC/WDPA/current/polygons')

# Print assets to show available layers and properties.
display(gfc)
print(wdpa.limit(10)) # Show first 10 records.

ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection.limit",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.loadTable",
          "arguments": {
            "tableId": {
              "constantValue": "WCMC/WDPA/current/polygons"
            }
          }
        }
      },
      "limit": {
        "constantValue": 10
      }
    }
  }
})


In [26]:
Map = geemap.Map()

# Display deforestation.
deforestation = gfc.select('lossyear')

Map.setOptions('SATELLITE')

Map.addLayer(deforestation, {
    "min": 1,
    "max": 20,
    "palette": ['yellow', 'orange', 'red']
}, 'Deforestation raster');

# Display WDPA data.
protectedArea = wdpa.filter(ee.Filter.equals('NAME', 'La Paya'))

# Display protected area as an outline (see F5.3 for paint()).
protectedAreaOutline = ee.Image().byte().paint(
    featureCollection = protectedArea,
    color = 1,
    width = 3
)

Map.addLayer(protectedAreaOutline, {
    "palette": 'white'
}, 'Protected area');

# Set up map display.
Map.centerObject(protectedArea)

Map

In [27]:
Map = geemap.Map()

# Convert from a deforestation raster to vector.
deforestationVector = deforestation.reduceToVectors(
    scale = deforestation.projection().nominalScale(),
    geometry = protectedArea.geometry(),
    labelProperty = 'lossyear', # Label polygons with a change year.
    maxPixels = 1e13
);

# Count the number of individual change events
# print('Number of change events:', deforestationVector.size())

# Display deforestation polygons. Color outline by change year.
deforestationVectorOutline = ee.Image().byte().paint(
    featureCollection = deforestationVector,
    color = 'lossyear',
    width = 1
);

Map.addLayer(deforestationVectorOutline, {
    "palette": ['yellow', 'orange', 'red'],
    "min": 1,
    "max": 20
}, 'Deforestation vector')


Map.centerObject(protectedArea)

Map

In [28]:
# # Generate deforestation point locations.
# deforestationCentroids = deforestationVector.map(function(feat) {
#     return feat.centroid()
# })

# Map.addLayer(deforestationCentroids, {
#     color: 'darkblue'
# }, 'Deforestation centroids');


# def calculate_deforestation_centroid(feature):
#     # Calcula el centroide de la geometría de la característica
#     return feature.geometry().centroid()

# # Aplica la función a cada elemento de la colección utilizando map
# deforestation_centroids = deforestationVector.map(calculate_deforestation_centroid)

# Imprime la colección resultante
# print(deforestation_centroids.getInfo())



In [29]:
Map = geemap.Map()

# Load required datasets.
gfc = ee.Image('UMD/hansen/global_forest_change_2020_v1_8')
wdpa = ee.FeatureCollection('WCMC/WDPA/current/polygons')

# Select a single protected area.
protectedArea = wdpa.filter(ee.Filter.equals('NAME', 'La Paya'))

# Maximum distance in meters is set in the brackets.
distance = protectedArea.distance(1000000)

Map.addLayer(distance, {
    "min": 0,
    "max": 20000,
    "palette": ['white', 'grey', 'black'],
    "opacity": 0.6
}, 'Distance');

Map.centerObject(protectedArea)

Map

In [30]:
Map = geemap.Map()


# Select a single protected area.
protectedArea = wdpa.filter(ee.Filter.equals('NAME', 'La Paya'))

def set_protected_attribute(feature):
    # Establece el atributo 'protected' en 1 y devuelve la característica modificada
    return feature.set('protected', 1)

# Aplica la función a cada elemento de la colección utilizando map
protected_area_raster = protectedArea.map(set_protected_attribute).reduceToImage(
    reducer=ee.Reducer.first(),
    selectors=['protected']
)



# Produce a raster of inside/outside the protected area.
# protectedAreaRaster = protectedArea.map(function(feat) {
#     return feat.set('protected', 1)
# }).reduceToImage(['protected'], ee.Reducer.first());

Map.addLayer(distance.updateMask(protectedAreaRaster), {
    "min": 0,
    "max": 20000}, 'Distance inside protected area')

# Map.addLayer(distance.updateMask(protectedAreaRaster.unmask().not()), {
#     "min": 0,
#     "max": 20000}, 'Distance outside protected area')

Map

EEException: Required argument (properties) missing to function: Collection.reduceToImage